In [1]:
import ee
import geemap.foliumap as geemap

# ee.Authenticate()
ee.Initialize()

In [2]:
def get_landsat_data(start_date, end_date, geometry):
    landsat_data = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2').select('SR_B[2-7]').filterDate(start_date, end_date).filterBounds(geometry)
    return landsat_data

fresno = ee.Geometry.Point([-120.0, 36.0])

landsat_jul = get_landsat_data('2019-07-01', '2019-08-01', fresno).first()
landsat_aug = get_landsat_data('2019-08-01', '2019-09-01', fresno).first()
landsat_sep = get_landsat_data('2019-09-01', '2019-10-01', fresno).first()

In [3]:
vis_params = {'min': 0, 'max': 30000, 'bands': ['SR_B5', 'SR_B4', 'SR_B3']}
modis_vis = {'min': 0.0, 'max': 100.0, 'palette': ['e1e4b4', '999d60', '2ec409', '0a4b06']}

m = geemap.Map(center=(36, -120), zoom=8)
m.addLayer(landsat_jul, vis_params, 'July')
m.addLayer(landsat_aug, vis_params, 'August')
m.addLayer(landsat_sep, vis_params, 'September')
m.addLayerControl()
m

In [4]:
cdl_orig = ee.ImageCollection('USDA/NASS/CDL').filterDate('2019-01-01', '2019-12-31').select('cropland').first().clip(landsat_sep.geometry())
cultivated = ee.ImageCollection('USDA/NASS/CDL').filterDate('2019-01-01', '2019-12-31').select('cultivated').first().clip(landsat_sep.geometry())
cdl_cultivated = cdl_orig.mask(cultivated.eq(2))
cdl = cdl_cultivated.reproject(crs=landsat_sep.projection(), scale=30)

landsat_jul = landsat_jul.mask(cultivated.eq(2))
landsat_aug = landsat_aug.mask(cultivated.eq(2))
landsat_sep = landsat_sep.mask(cultivated.eq(2))

In [5]:
area = ee.Image.pixelArea().addBands(cdl)
 
areas = area.reduceRegion(reducer=ee.Reducer.sum().group(groupField=1, groupName='class'), 
                          geometry=cdl.geometry(), scale=30, maxPixels=1e10)

class_areas = ee.List(areas.get('groups'))

def get_area(item):
    area_dict = ee.Dictionary(item)
    class_number = ee.Number(area_dict.get('class')).format()
    area = ee.Number(area_dict.get('sum')).divide(1e6).round()
    return ee.List([class_number, area])

class_area_lists = class_areas.map(get_area)
 
result = ee.Dictionary(class_area_lists.flatten())
sorted_keys = result.keys().sort(result.values())

n_classes = 6
class_keys = sorted_keys.getInfo()[-n_classes:]
class_keys

# tot = 0
# for each in sorted_keys.getInfo():
#     tot = tot + result.getInfo()[each]
#     print(each, result.getInfo()[each])
# print(tot)

['36', '2', '204', '69', '75', '61']

In [6]:
main_crops = cdl.expression("b('cropland') == 75 || b('cropland') == 69 || b('cropland') == 204 || b('cropland') == 2 || b('cropland') == 36")
cdl = cdl.mask(main_crops)

In [7]:
def get_indices(image):
    ndvi = image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')  #NIR, R
    ndwi = image.normalizedDifference(['SR_B5', 'SR_B6']).rename('NDWI')  #NIR, SWIR

    swir = image.select('SR_B7')
    mir = image.select('SR_B6')
    nir = image.select('SR_B5')
    red = image.select('SR_B4')
    green = image.select('SR_B3')
    blue = image.select('SR_B2')

    evi = ((nir.subtract(red)).divide(nir.add(red.multiply(6)).subtract(blue.multiply(7.5)).add(1))).multiply(2.5).rename('EVI')
    arvi = (nir.subtract(red.multiply(2)).add(blue)).divide(nir.add(red.multiply(2)).add(blue)).rename('ARVI')
    savi = ((nir.subtract(red)).divide(nir.add(red).add(0.5))).multiply(1.5).rename('SAVI')
    msi = mir.divide(nir).rename('MSI')
    
    gci = (nir.divide(green)).subtract(1).rename('GCI')
    nbri = image.normalizedDifference(['SR_B5', 'SR_B7']).rename('NBRI')
    
    return ndvi, ndwi, evi, arvi, savi, msi, gci, nbri

In [8]:
ndvi_jul, ndwi_jul, evi_jul, arvi_jul, savi_jul, msi_jul, gci_jul, nbri_jul = get_indices(landsat_jul.mask(main_crops))
ndvi_aug, ndwi_aug, evi_aug, arvi_aug, savi_aug, msi_aug, gci_aug, nbri_aug = get_indices(landsat_aug.mask(main_crops))
ndvi_sep, ndwi_sep, evi_sep, arvi_sep, savi_sep, msi_sep, gci_sep, nbri_sep = get_indices(landsat_sep.mask(main_crops))

In [9]:
def get_lai_fpar(start_date, end_date):
    lai = ee.ImageCollection('MODIS/061/MCD15A3H').filterDate(start_date, end_date).select('Lai').first().clip(landsat_sep.geometry())
    lai = lai.mask(main_crops)
    lai = lai.resample('bicubic').reproject(crs=landsat_sep.projection(), scale=30)
    
    fpar = ee.ImageCollection('MODIS/061/MCD15A3H').filterDate(start_date, end_date).select('Fpar').first().clip(landsat_sep.geometry())
    fpar = fpar.mask(main_crops)
    fpar = fpar.resample('bicubic').reproject(crs=landsat_sep.projection(), scale=30)
    
    return lai, fpar

In [10]:
lai_jul, fpar_jul = get_lai_fpar('2019-07-01', '2019-08-01')
lai_aug, fpar_aug = get_lai_fpar('2019-08-01', '2019-09-01')
lai_sep, fpar_sep = get_lai_fpar('2019-09-01', '2019-10-01')

In [11]:
vis_params = {'min': 0, 'max': 30000, 'bands': ['SR_B5', 'SR_B4', 'SR_B3']}
modis_vis = {'min': 0.0, 'max': 100.0, 'palette': ['e1e4b4', '999d60', '2ec409', '0a4b06']}

m = geemap.Map(center=(36, -120), zoom=8)
m.addLayer(landsat_jul.mask(main_crops), vis_params, 'July')
m.addLayer(landsat_aug.mask(main_crops), vis_params, 'August')
m.addLayer(landsat_sep.mask(main_crops), vis_params, 'September')
m.addLayer(ndvi_sep, {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'green']}, 'NDVI')
m.addLayer(ndwi_sep, {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'green']}, 'NDWI')
m.addLayer(evi_sep, {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'green']}, 'EVI')
m.addLayer(lai_sep, modis_vis, 'LAI')
m.addLayer(cdl, {}, 'CDL')
m.addLayer(main_crops, {}, 'Major crops')
m.addLayerControl()
m

In [12]:
def get_training_data(image, bands):
    data = image
    data = data.addBands(bands)
    return data

bands_jul = [ndvi_jul, ndwi_jul, evi_jul, arvi_jul, savi_jul, msi_jul, gci_jul, nbri_jul, lai_jul, fpar_jul]
bands_aug = [ndvi_aug, ndwi_aug, evi_aug, arvi_aug, savi_aug, msi_aug, gci_aug, nbri_aug, lai_aug, fpar_aug]
bands_sep = [ndvi_sep, ndwi_sep, evi_sep, arvi_sep, savi_sep, msi_sep, gci_sep, nbri_sep, lai_sep, fpar_sep]
    
data_jul = get_training_data(landsat_jul, bands_jul)
data_aug = get_training_data(landsat_aug, bands_aug)
data_sep = get_training_data(landsat_sep, bands_sep)

# data_sep.bandNames().getInfo()

In [13]:
points = cdl.stratifiedSample(region=landsat_sep.geometry(), scale=30, numPoints=3000, seed=0, geometries=True)

label = 'cropland'
bands = data_sep.bandNames().getInfo()

    
def classification(image):
    sample = image.select(bands).sampleRegions(collection=points, properties=[label], scale=30)
    sample = sample.randomColumn()
    split = 0.75

    training = sample.filter(ee.Filter.lt('random', split))
    validation = sample.filter(ee.Filter.gte('random', split))

    return training, validation

t_jul, v_jul = classification(data_jul)
t_aug, v_aug = classification(data_aug)
t_sep, v_sep = classification(data_sep)

training = t_jul.merge(t_aug).merge(t_sep)
validation = v_jul.merge(v_aug).merge(v_sep)

# m = geemap.Map(center=(36, -120), zoom=8)
# m.addLayer(cdl, {}, 'CDL')
# m.addLayer(points, {}, 'training')
# m.addLayerControl()
# m

In [14]:
classifier = ee.Classifier.smileRandomForest(128).train(training, label, bands)

train_accuracy = classifier.confusionMatrix()
train_accuracy.accuracy().getInfo()

0.9968375446696816

In [15]:
validated = validation.classify(classifier)

test_accuracy = validated.errorMatrix('cropland', 'classification')
test_accuracy.accuracy().getInfo()

0.7165922689714123